In [1]:
import pandas as pd
import re
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# Text Preprocessing Function
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    words = text.split()
    words = [lemmatizer.lemmatize(stemmer.stem(word)) for word in words if word not in stop_words]
    return ' '.join(words).strip()

# Load dataset
df = pd.read_csv("sentimentdataset.csv")
df = df[["Text", "Sentiment"]]
df["Sentiment"] = df["Sentiment"].str.strip()
df["ProcessedText"] = df["Text"].apply(preprocess_text)

# Map Sentiments into 3 Classes
positive_labels = {"positive", "good", "excellent", "great", "happy", "love", "amazing", "awesome", "fantastic", "satisfied"}
negative_labels = {"negative", "bad", "poor", "terrible", "hate", "angry", "disappointed", "worst", "awful", "frustrated"}

def categorize_sentiment(sentiment):
    sentiment = sentiment.lower().strip()
    if sentiment in positive_labels:
        return "positive"
    elif sentiment in negative_labels:
        return "negative"
    else:
        return "neutral"

df["Sentiment"] = df["Sentiment"].apply(categorize_sentiment)

# Encode labels
label_encoder = LabelEncoder()
df["SentimentEncoded"] = label_encoder.fit_transform(df["Sentiment"])

# Split dataset
X = df["ProcessedText"]
y = df["SentimentEncoded"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Convert text to numerical features using TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=30000, min_df=3, stop_words="english", sublinear_tf=True)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Apply SMOTE
min_class_size = y_train.value_counts().min()
k_neighbors = min(5, min_class_size - 1)
smote = SMOTE(random_state=42, k_neighbors=k_neighbors)
X_train_tfidf, y_train = smote.fit_resample(X_train_tfidf, y_train)

# Train MLP Classifier
best_mlp = MLPClassifier(
    activation='relu',
    alpha=0.001,
    batch_size=32,
    hidden_layer_sizes=(100, 100),
    learning_rate='constant',
    max_iter=500,
    solver='adam',
    random_state=42
)
best_mlp.fit(X_train_tfidf, y_train)

# Predict with MLP
y_pred = best_mlp.predict(X_test_tfidf)
print(f"MLP Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("MLP Classification Report:\n", classification_report(y_test, y_pred))

# Define DNN Model
def build_dnn_model(input_dim, num_classes):
    model = Sequential([
        Dense(512, activation='relu', input_shape=(input_dim,)),
        Dropout(0.3),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Prepare DNN input
y_train_dnn = to_categorical(y_train, num_classes=3)
y_test_dnn = to_categorical(y_test, num_classes=3)
input_dim = X_train_tfidf.shape[1]
dnn_model = build_dnn_model(input_dim, 3)
dnn_model.fit(X_train_tfidf.toarray(), y_train_dnn, epochs=10, batch_size=32, validation_split=0.1)

# Predict with DNN
y_pred_dnn = np.argmax(dnn_model.predict(X_test_tfidf.toarray()), axis=1)
print("DNN Accuracy:", accuracy_score(y_test, y_pred_dnn))
print("DNN Classification Report:\n", classification_report(y_test, y_pred_dnn))

2025-03-17 15:58:21.865104: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-17 15:58:21.880411: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-17 15:58:20.428929: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-17 15:58:20.632206: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742207300.789243    3455 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742207300.82

MLP Accuracy: 0.8435
MLP Classification Report:
               precision    recall  f1-score   support

           0       0.33      0.20      0.25         5
           1       0.91      0.92      0.92       130
           2       0.25      0.25      0.25        12

    accuracy                           0.84       147
   macro avg       0.50      0.46      0.47       147
weighted avg       0.84      0.84      0.84       147



/home/anurag/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-03-17 15:58:42.665940: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Epoch 1/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.4865 - loss: 0.9739 - val_accuracy: 0.9679 - val_loss: 0.4339
Epoch 2/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9599 - loss: 0.1474 - val_accuracy: 0.9936 - val_loss: 0.0294
Epoch 3/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9931 - loss: 0.0245 - val_accuracy: 1.0000 - val_loss: 0.0086
Epoch 4/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9957 - loss: 0.0104 - val_accuracy: 1.0000 - val_loss: 0.0107
Epoch 5/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9974 - loss: 0.0176 - val_accuracy: 1.0000 - val_loss: 0.0022
Epoch 6/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9967 - loss: 0.0138 - val_accuracy: 1.0000 - val_loss: 0.0014
Epoch 7/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9984 - loss: 0.0135 - val_accuracy: 1.0000 - val_loss: 0.0011
Epoch 8/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9992 - loss: 0.0060 - val_accuracy: 1.0000 - val_lo

In [5]:
import pandas as pd
import re
import nltk
import numpy as np
import spacy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics.pairwise import cosine_similarity

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# Load spaCy model
try:
    nlp = spacy.load("en_core_web_md")
except OSError:
    print("Downloading spaCy model...")
    import spacy.cli
    spacy.cli.download("en_core_web_md")
    nlp = spacy.load("en_core_web_md")

# Reference words for sentiment classification
ref_words = {
    "Positive": "positive",
    "Negative": "negative",
    "Neutral": "neutral"
}

# Compute reference vectors
ref_vectors = {category: nlp(word).vector for category, word in ref_words.items()}

# Function to categorize sentiment using cosine similarity
def assign_sentiment_category(sentiment):
    try:
        word_vector = nlp(sentiment).vector.reshape(1, -1)
        similarities = {category: cosine_similarity(word_vector, ref_vec.reshape(1, -1))[0][0] for category, ref_vec in ref_vectors.items()}
        return max(similarities, key=similarities.get).lower()
    except:
        return "neutral"  # Default to neutral if processing fails

# Text Preprocessing Function
def preprocess_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    words = text.split()
    words = [lemmatizer.lemmatize(stemmer.stem(word)) for word in words if word not in stop_words]
    return ' '.join(words).strip()

# Load dataset
df = pd.read_csv("sentimentdataset.csv")
df = df[["Text", "Sentiment"]].dropna()
df["Sentiment"] = df["Sentiment"].str.strip()

# Check unique sentiment classes before categorization
print("Unique sentiment values before categorization:", df["Sentiment"].unique())

# Apply preprocessing
df["ProcessedText"] = df["Text"].apply(preprocess_text)

# Apply sentiment classification using spaCy word embeddings
df["Sentiment"] = df["Sentiment"].apply(assign_sentiment_category)

# Check unique sentiment values after categorization
print("Unique sentiment values after categorization:", df["Sentiment"].unique())

# Encode labels
label_encoder = LabelEncoder()
df["SentimentEncoded"] = label_encoder.fit_transform(df["Sentiment"])

# Check class distribution before splitting
print("Class distribution before splitting:", df["SentimentEncoded"].value_counts())

# Ensure multiple classes exist
if df["SentimentEncoded"].nunique() < 2:
    print("Warning: Dataset contains only one class after preprocessing. Consider checking dataset integrity.")

# Split dataset
X = df["ProcessedText"]
y = df["SentimentEncoded"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Convert text to numerical features using TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=30000, min_df=3, stop_words="english", sublinear_tf=True)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Check class distribution after split
print("Class distribution in training set before SMOTE:", pd.Series(y_train).value_counts())

# Apply SMOTE only if there are at least 2 classes
if y_train.nunique() > 1:
    min_class_size = y_train.value_counts().min()
    k_neighbors = min(5, min_class_size - 1) if min_class_size > 1 else 1
    smote = SMOTE(random_state=42, k_neighbors=k_neighbors)
    X_train_tfidf, y_train = smote.fit_resample(X_train_tfidf, y_train)

# Check class distribution after SMOTE
print("Class distribution in training set after SMOTE:", pd.Series(y_train).value_counts())

# Train MLP Classifier
best_mlp = MLPClassifier(
    activation='relu',
    alpha=0.001,
    batch_size=32,
    hidden_layer_sizes=(100, 100),
    learning_rate='constant',
    max_iter=500,
    solver='adam',
    random_state=42
)
best_mlp.fit(X_train_tfidf, y_train)

# Predict with MLP
y_pred = best_mlp.predict(X_test_tfidf)
print(f"MLP Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("MLP Classification Report:\n", classification_report(y_test, y_pred))

# Define DNN Model
def build_dnn_model(input_dim, num_classes):
    model = Sequential([
        Dense(512, activation='relu', input_shape=(input_dim,)),
        Dropout(0.3),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Check again before DNN training
if y_train.nunique() < 2 or y_test.nunique() < 2:
    print("Warning: Dataset contains only one class after preprocessing. Model training may be invalid.")
else:
    y_train_dnn = to_categorical(y_train, num_classes=3)
    y_test_dnn = to_categorical(y_test, num_classes=3)
    input_dim = X_train_tfidf.shape[1]
    dnn_model = build_dnn_model(input_dim, 3)
    dnn_model.fit(X_train_tfidf.toarray(), y_train_dnn, epochs=10, batch_size=32, validation_split=0.1)
    y_pred_dnn = np.argmax(dnn_model.predict(X_test_tfidf.toarray()), axis=1)
    print("DNN Accuracy:", accuracy_score(y_test, y_pred_dnn))
    print("DNN Classification Report:\n", classification_report(y_test, y_pred_dnn))

[nltk_data] Downloading package stopwords to C:\Users\Valmik
[nltk_data]     Belgaonkar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Valmik
[nltk_data]     Belgaonkar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Unique sentiment values before categorization: ['Positive' 'Negative' 'Neutral' 'Anger' 'Fear' 'Sadness' 'Disgust'
 'Happiness' 'Joy' 'Love' 'Amusement' 'Enjoyment' 'Admiration' 'Affection'
 'Awe' 'Disappointed' 'Surprise' 'Acceptance' 'Adoration' 'Anticipation'
 'Bitter' 'Calmness' 'Confusion' 'Excitement' 'Kind' 'Pride' 'Shame'
 'Elation' 'Euphoria' 'Contentment' 'Serenity' 'Gratitude' 'Hope'
 'Empowerment' 'Compassion' 'Tenderness' 'Arousal' 'Enthusiasm'
 'Fulfillment' 'Reverence' 'Despair' 'Grief' 'Loneliness' 'Jealousy'
 'Resentment' 'Frustration' 'Boredom' 'Anxiety' 'Intimidation'
 'Helplessness' 'Envy' 'Regret' 'Curiosity' 'Indifference' 'Numbness'
 'Melancholy' 'Nostalgia' 'Ambivalence' 'Determination' 'Zest' 'Hopeful'
 'Proud' 'Grateful' 'Empathetic' 'Compassionate' 'Playful' 'Free-spirited'
 'Inspired' 'Confident' 'Bitterness' 'Yearning' 'Fearful' 'Apprehensive'
 'Overwhelmed' 'Jealous' 'Devastated' 'Frustrated' 'Envious' 'Dismissive'
 'Thrill' 'Bittersweet' 'Overjoyed' 'Insp